In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

In [2]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [3]:
def dataCollector(soup):
    import re
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[2]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    
    body = data[1:][1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    #tbl_d  = {name:col for name, col in zip(header,cols)}
    
    return body

In [4]:
dataCollector('https://en.wikipedia.org/wiki/The_Bachelorette_(season_2)')

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/The_Bachelorette_(season_2) (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x10b85df98>: Failed to establish a new connection: [Errno 65] No route to host',))

In [ ]:
def dataCollector(soup):
    import re
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[1]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    # create the header row and the body
    header = ['name','age', 'hometown', 'occupation', 'elimination']
    body = data[1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    tbl_d  = {name:col for name, col in zip(header,cols)}
    
    # extract the season number from the original soup
    number = re.findall(r'\d+', soup)
    
    # remove brackets
    num= ''.join(number)
    
    # create a new key of seasons
    tbl_d['season'] = [num] * len(tbl_d['age'])
    
    new_names = []
    for name in tbl_d['name']:
        cleaned_name = re.sub(r'\[\w+\]', ' ', name)
        new_names.append(cleaned_name)

    tbl_d['name'] = new_names
    
    # return dictionary
    return tbl_d

The code below is a way to try and find the special characteristics of the episode. Check back later on how this works.

In [5]:
def findBetween( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def findBetweenR( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [6]:
def removePrefix(text, prefix):
    return text[len(prefix):] if text.startswith(prefix) else text
        

In [48]:
def dataCollector(soup):
    import re
    import numpy as np
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[2]
    
    
    # find the descriptive colors in hex format
    descript_colors_cols = this.findChildren('dd')
    styles = [color.span['style'] for color in descript_colors_cols]
    colors = [findBetween(color,"background-color:",";") for color in styles]
    
    
    messy_color_codes = [dd.text for dd in descript_colors_cols]
    color_codes = [removePrefix(code,"\xa0\xa0\xa0\xa0 ") for code in messy_color_codes]
    
    color_dict = dict(zip(colors, color_codes))

    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    body = data[1:][1:]
    cols   =  zip(*body)
    
    body_n = []

    for i in body:
        if len(i) == len(body[0]):
            body_n.append(i)
        else:
            a = [" "]
            a.extend(" " * (len(body[0])))
            diff = len(a) - len(i) 
            n = i.extend( a[0:diff])
            body_n.append(n)
                
    # create a dict with the data
    #tbl_d  = {name:col for name, col in zip(header,cols)}
    contestants = [row[0] for row in body]
    
    #c2 = [row[2] for row in body]
    #ci2 = list(np.arange(1,len(c2) + 1))
    #c2dict = dict(zip(c2, ci2))
    
    #yup = [c2dict[x] for x in contestants]
    #yup = [c2dict.get(k, 'eliminated') for k in contestants] # default to eliminated if name doesn't exist
          
    return body

In [49]:
body = dataCollector('https://en.wikipedia.org/wiki/The_Bachelorette_(season_1)')

In [154]:
james = [1,2,3,4]
james.extend(" ")

TypeError: extend() takes exactly one argument (2 given)

In [162]:

james = ["j", "k", "z"]
james.extend(" " * 3)

TypeError: Can't convert 'int' object to str implicitly

In [44]:
body_n = []

for i in body:
    if len(i) == len(body[0]):
        body_n.append(i)
    else:
        a = [" "]
        a.extend(" " * (len(body[0])))
        diff = len(a) - len(i) 
        n = i.extend( a[0:diff])
        body_n.append(n)

In [50]:
body

[['Jamie', 'Russ', 'Charlie', 'Charlie', 'Ryan', 'Ryan', 'Ryan'],
 ['Rob', 'Greg T.', 'Bob', 'Russ', 'Charlie', 'Charlie', 'Charlie'],
 ['Chris', 'Ryan', 'Greg T.', 'Ryan', 'Russ', 'Russ', ''],
 ['Jack', 'Brian C.', 'Ryan', 'Greg T.', 'Greg T.', '', ' ', ' '],
 ['Brian S.', 'Brook', 'Mike', 'Bob', '', ' ', ' ', ' '],
 ['Eric', 'Mike', 'Rob', 'Jamie', '', ' ', ' ', ' '],
 ['Greg T.', 'Brian S.', 'Jamie', 'Mike', '', ' ', ' ', ' '],
 ['Matt', 'Jack', 'Russ', 'Rob', '', ' ', ' ', ' '],
 ['Ryan', 'Charlie', 'Brian C.', '', ' ', ' ', ' ', ' '],
 ['Brook', 'Josh', 'Brian S.', '', ' ', ' ', ' ', ' '],
 ['Peter', 'Brian H.', 'Brook', '', ' ', ' ', ' ', ' '],
 ['Brian H.', 'Jamie', 'Jack', '', ' ', ' ', ' ', ' '],
 ['Russ', 'Rob', 'Jeff', '', ' ', ' ', ' ', ' '],
 ['Paul', 'Jeff', 'Josh', '', ' ', ' ', ' ', ' '],
 ['Brian K.', 'Bob', '\nBrian H.\n', '', ' ', ' ', ' ', ' '],
 ['Bob', 'Billy', '', ' ', ' ', ' ', ' ', ' '],
 ['Billy', 'Brian K.', '', ' ', ' ', ' ', ' ', ' '],
 ['Duane', 'Chris', '

In [63]:
contestants = [row[0] for row in body]
l = []
for i in list(np.arange(0,(len(body[0])-1))):
    c = [row[i] for row in body]
    ci = list(np.arange(1,len(c) + 1))
    cdict = dict(zip(c, ci))
    yup = [cdict.get(k, 'eliminated') for k in contestants]
    l.append(yup)

In [64]:
l

[[1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25],
 [12,
  13,
  18,
  8,
  7,
  20,
  2,
  22,
  3,
  5,
  24,
  11,
  1,
  23,
  17,
  15,
  16,
  19,
  21,
  4,
  14,
  10,
  6,
  25,
  9],
 [7,
  6,
  'eliminated',
  12,
  10,
  'eliminated',
  3,
  'eliminated',
  4,
  11,
  'eliminated',
  'eliminated',
  8,
  'eliminated',
  'eliminated',
  2,
  'eliminated',
  'eliminated',
  'eliminated',
  9,
  13,
  14,
  5,
  'eliminated',
  1],
 [6,
  8,
  'eliminated',
  'eliminated',
  'eliminated',
  'eliminated',
  4,
  'eliminated',
  3,
  'eliminated',
  'eliminated',
  'eliminated',
  2,
  'eliminated',
  'eliminated',
  5,
  'eliminated',
  'eliminated',
  'eliminated',
  'eliminated',
  'eliminated',
  'eliminated',
  7,
  'eliminated',
  1],
 ['eliminated',
  'eliminated',
  'eliminated',
  'eliminated',
  'eliminated',
  'eliminated',
  4,
  'eliminated',
  1,
  'eliminated',
  'eliminated